##  🍫 [Day 19](https://adventofcode.com/2018/day/19)

In [0]:
from functools import partial

"""Defines all instructions (Reusing Day 16 code)"""
def apply_add(a, b, c, before, register=0):
  before[c] = before[a] + (before[b] if register else b)
  return before


def apply_mul(a, b, c, before, register=0):
  before[c] = before[a] * (before[b] if register else b)
  return before


def apply_ban(a, b, c, before, register=0):
  before[c] = before[a] & (before[b] if register else b)
  return before


def apply_bor(a, b, c, before, register=0):
  before[c] = before[a] | (before[b] if register else b)
  return before


def apply_set(a, b, c, before, register=0):
  before[c] = before[a] if register else a
  return before


def apply_gt(a, b, c, before, register=0):
  if register == 0:
    before[c] = a > before[b]
  elif register == 1:
    before[c] = before[a] > b
  else:
    before[c] = before[a] > before[b]    
  return before


def apply_eq(a, b, c, before, register=0):
  if register == 0:
    before[c] = a == before[b]
  elif register == 1:
    before[c] = before[a] == b
  else:
    before[c] = before[a] == before[b]    
  return before


"""Gather all instructions"""
operations = {'addr': partial(apply_add, register=1),
              'addi': partial(apply_add, register=0),
              'mulr': partial(apply_mul, register=1), 
              'muli': partial(apply_mul, register=0),
              'banr': partial(apply_ban, register=1), 
              'bani': partial(apply_ban, register=0),
              'borr': partial(apply_bor, register=1), 
              'bori': partial(apply_bor, register=0),
              'setr': partial(apply_set, register=1), 
              'seti': partial(apply_set, register=0),
              'gtir': partial(apply_gt, register=0), 
              'gtri': partial(apply_gt, register=1),
              'gtrr': partial(apply_gt, register=2),
              'eqir': partial(apply_eq, register=0), 
              'eqri': partial(apply_eq, register=1),
              'eqrr': partial(apply_eq, register=2)}

In [0]:
def run_programm(inputs, init_registers=None, verbose=False):
  """Run the background program"""
  registers = [0] * 6
  if init_registers is not None:
    registers = init_registers
  inst_register = inputs[0][1]
  instructions = inputs[1:]
  step = 0
  while 1:
    op, a, b, c = instructions[registers[inst_register]]
    operations[op](a, b, c, registers)
    if registers[inst_register] >= len(instructions) - 1:
      break
    registers[inst_register] += 1
    if verbose:
      print('Step %d' % (step + 1), op, a, b, c, '-->', registers)
      step += 1
  return registers

In [4]:
%%time
with open("day19.txt", 'r') as f:  
  inputs = f.read().splitlines()
  for i, line in enumerate(inputs):
    if line.startswith('#'):
      inputs[i] = ('inst', int(line[-1]), 0, 0)
    else:
      aux = line.split()
      inputs[i] = (aux[0], int(aux[1]), int(aux[2]), int(aux[3]))
      
print('Final value of register 0:', run_programm(inputs)[0])

Final value of register 0: 1140
CPU times: user 5.72 s, sys: 9.15 ms, total: 5.73 s
Wall time: 7.12 s


For Part II, running the program for a few instructions show that there is a cyclic structure in the program... In my input, the register state at `step 28` is `[0, 1, 10, 0, 2, 10551331]`,  and it is where the program starts to loop through instructions 13, 5, 6, 7, 8, 10, 11, 12. 
```

In [6]:
_ = run_programm(inputs, init_registers=[1, 0, 0, 0, 0, 0], verbose=True)

Step 1 addi 2 16 2 --> [1, 0, 17, 0, 0, 0]
Step 2 addi 5 2 5 --> [1, 0, 18, 0, 0, 2]
Step 3 mulr 5 5 5 --> [1, 0, 19, 0, 0, 4]
Step 4 mulr 2 5 5 --> [1, 0, 20, 0, 0, 76]
Step 5 muli 5 11 5 --> [1, 0, 21, 0, 0, 836]
Step 6 addi 3 4 3 --> [1, 0, 22, 4, 0, 836]
Step 7 mulr 3 2 3 --> [1, 0, 23, 88, 0, 836]
Step 8 addi 3 7 3 --> [1, 0, 24, 95, 0, 836]
Step 9 addr 5 3 5 --> [1, 0, 25, 95, 0, 931]
Step 10 addr 2 0 2 --> [1, 0, 27, 95, 0, 931]
Step 11 setr 2 1 3 --> [1, 0, 28, 27, 0, 931]
Step 12 mulr 3 2 3 --> [1, 0, 29, 756, 0, 931]
Step 13 addr 2 3 3 --> [1, 0, 30, 785, 0, 931]
Step 14 mulr 2 3 3 --> [1, 0, 31, 23550, 0, 931]
Step 15 muli 3 14 3 --> [1, 0, 32, 329700, 0, 931]
Step 16 mulr 3 2 3 --> [1, 0, 33, 10550400, 0, 931]
Step 17 addr 5 3 5 --> [1, 0, 34, 10550400, 0, 10551331]
Step 18 seti 0 9 0 --> [0, 0, 35, 10550400, 0, 10551331]
Step 19 seti 0 8 2 --> [0, 0, 1, 10550400, 0, 10551331]
Step 20 seti 1 4 1 --> [0, 1, 2, 10550400, 0, 10551331]
Step 21 seti 1 2 4 --> [0, 1, 3, 10550400,

In other words, the loop is:

```
seti 2 7 2 --> Set R2 to 2
mulr 1 4 3 --> R3 = R1 * R4 = R4
eqrr 3 5 3 --> Set R3 to bool(R5 == R3)
addr 3 2 2 --> R2 += R3
  (addi 2 1 2 --> R2 += 1) if R3 = 0
  (addr 1 0 0 --> R0 += R1) if R3 = 1
addi 4 1 4 --> R4 += 1
gtrr 4 5 3 --> Set R3 to bool(R4 > R5)
addr 2 3 2 --> R2 += R3 (escape the loop when R3 is True)
```

which translates as:

In [0]:
def inner_loop(registers):
  # when R3 == R5 is possible, R0 gets updated once (when R4 = R5 // R1)
  # Oterwise the loop only increment R4 until it reaches R5
  if registers[5] % registers[1] == 0:
    registers[0] += registers[1] 
  registers[4] = registers[5] + 1
  registers[2] = 12

Replacing the instructions loop with this shortcut everytime I encouter instruction `seti 2 7 2`, I again run the program and detect another loop :[ , which is as follows:

```
addi 1 1 1 --> R1 += 1
gtrr 1 5 3 --> Set R3 to bool(R1 > R5)
addr 3 2 2 --> R2 += R3 (escape the loop if R3 is True)
seti 1 0 2 --> R2 = 1
seti 1 2 4 --> R4 = 1
  [... back to the first loop .... ]
```

In [0]:
 def main_loop(registers):
  """Outer loop shortcut"""
  while registers[1] <= registers[5]:
    registers[4] = 1
    inner_loop(registers)
    registers[1] += 1
  registers[2] = 16
  
def run_programm_with_loop(inputs, init_registers=None):
  """Run the background program ut shortcut the inner loop"""
  registers = [0] * 6
  if init_registers is not None:
    registers = init_registers
  inst_register = inputs[0][1]
  instructions = inputs[1:]
  while 1:
    # First inner loop
    if registers[inst_register] == 11:
      inner_loop(registers)
    # Outer loop
    if registers[inst_register] == 13:
      main_loop(registers)
    else:
      op, a, b, c = instructions[registers[inst_register]]
      operations[op](a, b, c, registers)
      if registers[inst_register] >= len(instructions) - 1:
        break
      registers[inst_register] += 1
  return registers

In [10]:
%%time
print('Final value of register 0:', run_programm_with_loop(inputs, init_registers=[1, 0, 0, 0, 0, 0])[0])

Final value of register 0: 12474720
CPU times: user 5.45 s, sys: 2.77 ms, total: 5.45 s
Wall time: 5.45 s
